Conditional Edge（条件边）能够根据预设的条件来决定数据流向哪个节点。
LangGraph还有一个更强大的功能-Send，它能够在运行时动态地创建边，并传递自定义的状态，实现Map-Reduce这种高效的数据处理模式。
### 1.什么是Send
Send是LangGraph提供的一个特殊对象，它主要用于在条件边的路由函数中实现动态工作流控制。
与普通条件边只能返回单个目标节点名称（字符串）不同，Send允许返回一个Send对象列表，每个Send对象都制定了一个目标节点和要传递给该节点的自定义状态。
这样就可以在运行时根据数据内容动态地创建任意数量的边，实现更高效的并行处理。
简单来说，Send可以让工作流能够根据实际数据灵活地决定要执行哪些节点、执行多少次、以及每次执行时传递什么状态，从而突破了传统条件边必须在定义时就固定数量的限制。
### 2.为什么需要Send
设想一个场景，有一个初始节点，它生成一个主题列表，比如["天气","股票","新闻"]。
每个主题都需要被不同的节点处理，比如一个节点负责处理天气，一个节点负责处理股票，一个节点负责处理新闻。
传统的条件边只能根据固定的条件（比如主题名称）来决定流向哪个节点，无法根据实际数据动态调整。
我们需要的是一种更高效的数据处理模式：Map-Reduce:
1. Map阶段：将一个大数据集拆分成多个小的数据项，然后对每个数据项应用相同的处理函数。
2. Reduce阶段：将Map阶段的处理结果合并起来，得到最终的结果。
在LangGraph中，可以使用Send来实现Map-Reduce模式：
1. 并行执行：所有Send对象指向的节点会并行执行（如果支持的话）
2. 状态传递：每个Send对象携带的状态会传递给对应的节点
3. 结果合并：各个节点返回的状态更新会根据Reducer规则合并到主状态中
### 3.Send的基本使用
#### 3.1 Send对象的定义与基本写法
在LangGraph中，Send是一个类，它有两个参数：
- node：目标节点的名称（字符串）
- args：要传递给目标节点的状态：{"状态键": "状态值"}
基本写法：
```python
from langgraph.types import Send

Send("节点名称", {"状态键": "状态值"})
```
#### 3.2 在条件边中使用Send
Send通常用在条件边的路由函数中。与普通条件边返回字符串不同，使用Send条件边可以返回一个Send对象列表。
下面用一个例子来演示：

In [1]:
from typing import TypedDict, Annotated
import operator
from langgraph.graph import StateGraph, START, END
from langgraph.types import Send

# 定义state
class AppState(TypedDict):
    subjects: list[str] # 主题列表
    jokes: Annotated[list[str], operator.add] # 笑话列表,shiy operator.add作为Reducer

# 定义处理单个主题的节点函数
def generate_joke(state: AppState) -> AppState:
    """为单个subject生成笑话"""
    # 注意：Send传递的是单数subject，不是subjects列表
    subject = state["subject"]
    joke = f"关于{subject}的笑话：为什么{subject}总是那么可爱？因为它们天生就是！"
    # 返回一个包含单个笑话的列表
    return {"jokes": [joke]}

# 定义路由函数
# 遍历subjects列表中的每个主题，为每个主题创建一个Send对象。
# 每个Send对象都指定了目标节点"generate_joke"和一个包含subject的参数字典。
def continue_to_jokes(state: AppState):
    """为每个主题创建Send对象，动态发送到generate_joke节点"""
    return [
        Send("generate_joke", {"subject": subject})
        for subject in state["subjects"]
    ]

# 构建Graph
graph = StateGraph(AppState)
graph.add_node("generate_joke", generate_joke)
# 从START开始，使用条件边动态创建Send
graph.add_conditional_edges(START, continue_to_jokes)
graph.add_edge("generate_joke", END)
app = graph.compile()

result = app.invoke({"subjects": ["猫", "狗", "鸟"]})
print(result)

{'subjects': ['猫', '狗', '鸟'], 'jokes': ['关于猫的笑话：为什么猫总是那么可爱？因为它们天生就是！', '关于狗的笑话：为什么狗总是那么可爱？因为它们天生就是！', '关于鸟的笑话：为什么鸟总是那么可爱？因为它们天生就是！']}


### 4.使用Send的注意事项
#### 4.1 Reducer的选择
当多个Send并行执行时，每个节点都会返回状态更新。如果没有正确设置Reducer，可能会导致状态更新冲突或丢失。
通常，对于列表类型的字段，通常使用合并操作（+或extend）；对于数值类型，可以使用累加（add）；对于字典类型，需要自定义合并逻辑。
#### 4.2 状态传递的完整性
Send可以传递部分状态，但要注意目标节点可能需要访问其他手段。如果目标节点需要访问完整的State，可以在Send中传递完整状态。
#### 4.3 空列表的处理
如果条件边的路由函数返回空列表（即没有Send对象），Graph会直接结束执行，不会报错。
### 5.什么时候使用Send
Send适合将数据分发到同一个节点进行并行处理的场景，类似于编程中的for循环，但支持并行执行。
典型场景：
- Map-Reduce模式：将列表中的每个元素都应用相同的处理逻辑
- 批量处理：对一批数据进行相同的处理
- 动态工作流：根据运行时数据动态决定要创建多少个任务